In [17]:
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dropout
from keras.layers import Activation, BatchNormalization, Add, Reshape, DepthwiseConv2D
from keras.utils.vis_utils import plot_model
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K
import numpy as np
from keras.utils import np_utils

In [4]:
np.random.seed(1230)

In [27]:

def _make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


def relu6(x):
    """Relu 6
    """
    return K.relu(x, max_value=6.0)


def _conv_block(inputs, filters, kernel, strides):
    """Convolution Block
    This function defines a 2D convolution operation with BN and relu6.
    # Arguments
        inputs: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        strides: An integer or tuple/list of 2 integers,
            specifying the strides of the convolution along the width and height.
            Can be a single integer to specify the same value for
            all spatial dimensions.
    # Returns
        Output tensor.
    """

    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1

    x = Conv2D(filters, kernel, padding='same', strides=strides)(inputs)

    x = BatchNormalization(axis=channel_axis)(x)
    #x = BatchNormalizationF16(axis=channel_axis)(x)
    return LeakyReLU(alpha=0.03)(x)


def _bottleneck(inputs, filters, kernel, t, alpha, s, r=False):
    """Bottleneck
    This function defines a basic bottleneck structure.
    # Arguments
        inputs: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        t: Integer, expansion factor.
            t is always applied to the input size.
        s: An integer or tuple/list of 2 integers,specifying the strides
            of the convolution along the width and height.Can be a single
            integer to specify the same value for all spatial dimensions.
        alpha: Integer, width multiplier.
        r: Boolean, Whether to use the residuals.
    # Returns
        Output tensor.
    """

    channel_axis = 1 if K.image_data_format() == 'channels_first' else -1
    # Depth
    tchannel = K.int_shape(inputs)[channel_axis] * t
    # Width
    cchannel = int(filters * alpha)

    x = _conv_block(inputs, tchannel, (1, 1), (1, 1))

    x = DepthwiseConv2D(kernel, strides=(s, s), depth_multiplier=1, padding='same')(x)
    
    x = BatchNormalization(axis=channel_axis)(x)
    #x = BatchNormalizationF16(axis=channel_axis)(x)
    
    x = LeakyReLU(alpha=0.03)(x)

    x = Conv2D(cchannel, (1, 1), strides=(1, 1), padding='same')(x)
    
    x = BatchNormalization(axis=channel_axis)(x)
    #x = BatchNormalizationF16(axis=channel_axis)(x)
    
    if r:
        x = Add()([x, inputs])

    return x


def _inverted_residual_block(inputs, filters, kernel, t, alpha, strides, n):
    """Inverted Residual Block
    This function defines a sequence of 1 or more identical layers.
    # Arguments
        inputs: Tensor, input tensor of conv layer.
        filters: Integer, the dimensionality of the output space.
        kernel: An integer or tuple/list of 2 integers, specifying the
            width and height of the 2D convolution window.
        t: Integer, expansion factor.
            t is always applied to the input size.
        alpha: Integer, width multiplier.
        s: An integer or tuple/list of 2 integers,specifying the strides
            of the convolution along the width and height.Can be a single
            integer to specify the same value for all spatial dimensions.
        n: Integer, layer repeat times.
    # Returns
        Output tensor.
    """

    x = _bottleneck(inputs, filters, kernel, t, alpha, strides)

    for i in range(1, n):
        x = _bottleneck(x, filters, kernel, t, alpha, 1, True)

    return x


def MobileNetv2(input_shape, k, alpha=1.0):
    """MobileNetv2
    This function defines a MobileNetv2 architectures.
    # Arguments
        input_shape: An integer or tuple/list of 3 integers, shape
            of input tensor.
        k: Integer, number of classes.
        alpha: Integer, width multiplier, better in [0.35, 0.50, 0.75, 1.0, 1.3, 1.4].
    # Returns
        MobileNetv2 model.
    """
    inputs = Input(shape=input_shape)

    #first_filters = _make_divisible(32 * alpha, 8)
    first_filters = 32
    x = _conv_block(inputs, first_filters, (3, 3), strides=(2, 2))

    x = _inverted_residual_block(x, 16, (3, 3), t=2, alpha=alpha, strides=1, n=1)
    x = _inverted_residual_block(x, 32, (3, 3), t=2, alpha=alpha, strides=2, n=1)
    x = _inverted_residual_block(x, 64, (3, 3), t=2, alpha=alpha, strides=2, n=1)
    #x = _inverted_residual_block(x, 96, (3, 3), t=3, alpha=alpha, strides=1, n=2)
    x = _inverted_residual_block(x, 128, (3, 3), t=2, alpha=alpha, strides=2, n=1)
    #x = _inverted_residual_block(x, 320, (3, 3), t=6, alpha=alpha, strides=1, n=1)

    #if alpha > 1.0:
        #last_filters = _make_divisible(1280 * alpha, 8)
    #else:
        #last_filters = 1280
    last_filters = 64
    x = _conv_block(x, last_filters, (1, 1), strides=(1, 1))
    x = GlobalAveragePooling2D()(x)
    x = Reshape((1, 1, last_filters))(x)
    x = Dropout(0.3, name='Dropout')(x)
    x = Conv2D(k, (1, 1), padding='same')(x)

    x = Activation('softmax', name='softmax')(x)
    output = Reshape((k,))(x)

    model = Model(inputs, output)
    # plot_model(model, to_file='images/MobileNetv2.png', show_shapes=True)

    return model


if __name__ == '__main__':
    model = MobileNetv2((48, 48, 1), 7, 1.0)
    print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 48, 48, 1)         0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 24, 24, 32)        320       
_________________________________________________________________
batch_normalization_29 (Batc (None, 24, 24, 32)        128       
_________________________________________________________________
leaky_re_lu_21 (LeakyReLU)   (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 24, 24, 64)        2112      
_________________________________________________________________
batch_normalization_30 (Batc (None, 24, 24, 64)        256       
_________________________________________________________________
leaky_re_lu_22 (LeakyReLU)   (None, 24, 24, 64)        0         
__________

In [31]:
x = np.load("img_train.npy")
y = np.load("img_label.npy")

x_train = x[0:int(28709*0.8)]/255
y_train = y[0:int(28709*0.8)]
x_val = x[int(28709*0.8): 28709]/255
y_val = y[int(28709*0.8):28709]

In [28]:
model.compile(loss='categorical_crossentropy',
              optimizer= 'Adam',
              metrics=['accuracy'])
#print(MODEL.model.summary())

In [29]:
model.save("test.h5")

In [32]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.3,
    zoom_range=[0.8, 1.2],
    horizontal_flip=True,
    vertical_flip=True
    )

datagen.fit(x_train)
train_generator = datagen.flow(x_train,y_train,batch_size = 200)
#learning_rate_function = ReduceLROnPlateau(monitor='val_acc',patience=2,epsilon=0.00001,verbose=1,factor=0.2)
train_history2 = model.fit_generator(train_generator, steps_per_epoch=1500,epochs=25,verbose=1,validation_data=(x_val,y_val))

Epoch 1/25
1500/1500 [==============================] - 129s 86ms/step - loss: 1.7331 - acc: 0.2924 - val_loss: 1.6242 - val_acc: 0.3577
Epoch 2/25
1500/1500 [==============================] - 124s 83ms/step - loss: 1.5652 - acc: 0.3839 - val_loss: 1.5946 - val_acc: 0.3575
Epoch 3/25
1500/1500 [==============================] - 124s 83ms/step - loss: 1.4674 - acc: 0.4325 - val_loss: 1.4649 - val_acc: 0.4349
Epoch 4/25
1500/1500 [==============================] - 125s 83ms/step - loss: 1.4122 - acc: 0.4564 - val_loss: 1.4798 - val_acc: 0.4225
Epoch 5/25
1500/1500 [==============================] - 124s 83ms/step - loss: 1.3791 - acc: 0.4709 - val_loss: 1.3172 - val_acc: 0.4937
Epoch 6/25
1500/1500 [==============================] - 123s 82ms/step - loss: 1.3521 - acc: 0.4836 - val_loss: 1.4717 - val_acc: 0.4347
Epoch 7/25
1500/1500 [==============================] - 123s 82ms/step - loss: 1.3340 - acc: 0.4902 - val_loss: 1.2831 - val_acc: 0.5045
Epoch 8/25
1500/1500 [===================

In [8]:
model_no = MobileNetv2((48, 48, 1), 7, 1.0)
print(model_no.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 48, 48, 1)         0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 24, 24, 32)        320       
_________________________________________________________________
batch_normalization_15 (Batc (None, 24, 24, 32)        128       
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 24, 24, 32)        1056      
_________________________________________________________________
batch_normalization_16 (Batc (None, 24, 24, 32)        128       
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 24, 24, 32)        0         
__________

In [33]:
model_tmp = model

In [35]:
epochs = 100
batch_size = 2000
model_no.compile(loss='categorical_crossentropy',
              optimizer= 'Adam',
              metrics=['accuracy'])
model_no.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
        validation_data=(x_val,y_val))
        #callbacks=[learning_rate_function])
#score = MODEL.model.evaluate(x_test, y_test, verbose=0)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])
#model_no.save('MobileNet.h5')

Train on 22967 samples, validate on 5742 samples
Epoch 1/100
22967/22967 [==============================] - 18s 774us/step - loss: 1.9753 - acc: 0.1806 - val_loss: 1.8669 - val_acc: 0.2461
Epoch 2/100
22967/22967 [==============================] - 7s 322us/step - loss: 1.8165 - acc: 0.2635 - val_loss: 1.7671 - val_acc: 0.2886
Epoch 3/100
22967/22967 [==============================] - 7s 318us/step - loss: 1.7452 - acc: 0.3000 - val_loss: 1.7489 - val_acc: 0.3042
Epoch 4/100
22967/22967 [==============================] - 7s 316us/step - loss: 1.6954 - acc: 0.3232 - val_loss: 1.7535 - val_acc: 0.3116
Epoch 5/100
22967/22967 [==============================] - 7s 320us/step - loss: 1.6489 - acc: 0.3472 - val_loss: 1.7090 - val_acc: 0.3276
Epoch 6/100
22967/22967 [==============================] - 7s 320us/step - loss: 1.6115 - acc: 0.3620 - val_loss: 1.6926 - val_acc: 0.3400
Epoch 7/100
22967/22967 [==============================] - 7s 320us/step - loss: 1.5731 - acc: 0.3853 - val_loss: 1.

KeyboardInterrupt: 

In [37]:
model.save("MobileNetv2.h5")
model.save_weights("Weightv2.h5")

In [2]:
import csv
n = 7178
test_x = []
for i in range(n):
    test_x.append([])

count = 0
with open('test.csv', newline='') as csvfile:
    rows = csv.reader(csvfile)
    for row in rows:
        if(row[0] != "id"):
            test_x[count].append(row[1])
            count += 1
x_test = []
for i in range(len(test_x)):
    x_test.append(test_x[i][0].split(' '))

for i in range(len(test_x)):
    for j in range(len(x_test[0])):
        x_test[i][j] = float(x_test[i][j])
x_test = np.array(x_test)
x_test = x_test.reshape(7178,48,48,1)
x_test = x_test/255

In [3]:
hw3_model = load_model("hw3_model.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [4]:
hw3_predict = hw3_model.predict_classes(x_test)

In [6]:
hw3_predict.shape

(7178,)

In [18]:
hw3_y = np_utils.to_categorical(hw3_predict)

In [35]:
epochs = 100
batch_size = 3000
#model_no.compile(loss='categorical_crossentropy',
#              optimizer= 'Adam',
#              metrics=['accuracy'])
model_tmp.fit(x_test, hw3_y,
          batch_size=batch_size,
          epochs=epochs)
        #callbacks=[learning_rate_function])
#score = MODEL.model.evaluate(x_test, y_test, verbose=0)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])
#model_no.save('MobileNet.h5')

Epoch 1/100
7178/7178 [==============================] - 2s 268us/step - loss: 0.5212 - acc: 0.8154
Epoch 2/100
7178/7178 [==============================] - 2s 254us/step - loss: 0.5176 - acc: 0.8136
Epoch 3/100
7178/7178 [==============================] - 2s 263us/step - loss: 0.5160 - acc: 0.8140
Epoch 4/100
7178/7178 [==============================] - 2s 258us/step - loss: 0.5168 - acc: 0.8139
Epoch 5/100
7178/7178 [==============================] - 2s 257us/step - loss: 0.5145 - acc: 0.8213
Epoch 6/100
7178/7178 [==============================] - 2s 255us/step - loss: 0.5042 - acc: 0.8256
Epoch 7/100
7178/7178 [==============================] - 2s 256us/step - loss: 0.5038 - acc: 0.8243
Epoch 8/100
7178/7178 [==============================] - 2s 260us/step - loss: 0.5025 - acc: 0.8235
Epoch 9/100
7178/7178 [==============================] - 2s 259us/step - loss: 0.4952 - acc: 0.8288
Epoch 10/100
7178/7178 [==============================] - 2s 258us/step - loss: 0.4992 - acc: 0.8218

In [36]:
prediction = model_tmp.predict(x_test)

In [37]:
prediction2 = np.argmax(prediction,axis=1)

In [38]:
prediction2

array([3, 4, 3, ..., 3, 0, 2])

In [ ]:
model_tmp.save("")

In [39]:
with open("submission.csv", 'w', newline='') as csvfile:
    csvfile.write('id,label\n')
    for i, v in enumerate(prediction2):
        csvfile.write('%d,%d\n' %(i, prediction2[i]))